In [5]:
import telebot
import bs4
import requests
import re
import pandas as pd

with open('token.txt') as fh:
    token = fh.read().strip()
bot = telebot.TeleBot(token)
# Изначально ничего не запарсили
parsed = False
# Поддерживаемые страны
supported = ['France', 'China', 'Italy', 'Russia']

# Команды
@bot.message_handler(commands=['start'])
def show_start(message):
    bot.send_message(message.from_user.id, "Добрый день. Я умею работать с сайтом https://www.worldometers.info/coronavirus/.\
Если вы введете название страны на английском языке я выведу вам статистику с 15 февраля: сколько было \
зафиксировано новых случаев и смертей за каждый из день. После того, как я соберу данные, я смогу вывести следующую информацию: \
ввывести среднее количество случаев или смертей (по запросу) за это время, вывести количество новых случаев или смертей за \
любую дату, вывести количество дней, в которые было зафиксировано больше случаев или смертей, чем в предыдущие. \
Чтобы посмотреть все команды нажмите /help. Для начала парсинга нажмите /parse. Для просмотра доступных стран для парсинга нажмите /parse_help.")

@bot.message_handler(commands=['help'])
def show_help(message):
    bot.send_message(message.from_user.id,"/parse - ввести страну и запустить парсинг\n/parse_help - вывести список поддерживаемых стран\
    \n/file - получить файл с данными\n/median - посчитать медиану для выбранной колонки\
    \n/mean - посчитать среднее для выбранной колонки\n/date - получить информацию по конкретному дню")
    
@bot.message_handler(commands=['parse'])
def parse(message):
    global parsed
    parsed = False
    bot.send_message(message.from_user.id, "Введите название страны на английском языке: ")
    
@bot.message_handler(commands=['parse_help'])
def show_parse_help(message):
    bot.send_message(message.from_user.id, f"Пока я могу собрать информацию только для этих стран:\n {' '.join(supported)}")
    
@bot.message_handler(commands=['file'])
def get_file(message):
    global parsed
    if parsed:
        fh = open('data.csv', 'rb')
        bot.send_document(message.from_user.id, fh)
        fh.close()
    else:
        bot.send_message(message.from_user.id, "Парсинг не выполнен. Нажмите /parse чтобы это сделать")
        
@bot.message_handler(commands=['median'])
def get_median(message):
    global parsed
    if parsed:
        col = message.text.split()
        if len(col) == 2:
            if col[1] == 'cases':
                col = 'number of daily cases'
            else:
                col = 'number of daily deaths'
        else:
            col = 'number of daily cases'
        
        data = pd.read_csv('data.csv', delimiter = ',')
        med = data[col].median()
        bot.send_message(message.from_user.id, "Медиана для колонки " + col + " = " + str(med))
    else:
        bot.send_message(message.from_user.id, "Парсинг не выполнен. Нажмите /parse чтобы это сделать")

@bot.message_handler(commands=['mean'])
def get_median(message):
    global parsed
    if parsed:
        col = message.text.split()
        if len(col) == 2:
            if col[1] == 'cases':
                col = 'number of daily cases'
            else:
                col = 'number of daily deaths'
        else:
            col = 'number of daily cases'
            
        data = pd.read_csv('data.csv', delimiter = ',')
        mea = data[col].mean()
        bot.send_message(message.from_user.id, "Среднее для колонки " + col + " = " + str(mea))
    else:
        bot.send_message(message.from_user.id, "Парсинг не выполнен. Нажмите /parse чтобы это сделать")
        
@bot.message_handler(commands=['date'])
def get_date(message):
    global parsed
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul']
    if parsed:
        col = message.text.split()
        if len(col) != 3:
            bot.send_message(message.from_user.id, "Дата не указана.")
        print(col)
        mon = col[1]
        day = col[2]
        if mon in months and int(day) >= 1 and int(day) <= 31:
            data = pd.read_csv('data.csv', delimiter = ',')
            bot.send_message(message.from_user.id, f'{mon + " " + day} было зарегестрировано {data[data.date == mon + " " + day]["number of daily cases"].values[0]} случаев')
        else:
            bot.send_message(message.from_user.id, "Ошибка в дате, попробуйте еще раз.")
    else:
        bot.send_message(message.from_user.id, "Парсинг не выполнен. Нажмите /parse чтобы это сделать")
    
# Текст
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    global parsed
    if not parsed:
        if message.text in supported:
            try:
                bot.send_message(message.from_user.id, "Начинаю парсинг. Подождите...")
                
                url = f'https://www.worldometers.info/coronavirus/country/{message.text.lower()}'

                html = requests.get(url).text
                soup = bs4.BeautifulSoup(html, 'lxml')

                dates_cases = []
                cases = []

                for graph in soup.find_all('script', {'type': "text/javascript"}):
                    if 'Daily New Cases' in str(graph):
                        dates_cases = re.findall(r'categories: \[([\w\s",]+)', str(graph))[0]
                        cases = re.findall(r'data: \[([\w,]+)', str(graph))[0]

                dates_deaths = []
                deaths = []

                for graph in soup.find_all('script', {'type': "text/javascript"}):
                    if 'Daily Deaths' in str(graph):
                        dates_deaths = re.findall(r'categories: \[([\w\s",]+)', str(graph))[0]
                        deaths = re.findall(r'data: \[([\w,]+)', str(graph))[0]

                dates = [date.strip('"') for date in dates_cases.split(',')]
                cases = cases.replace('null', '0')
                deaths = deaths.replace('null', '0')
                cases = [int(x) for x in cases.split(',')]
                deaths = [int(x) for x in deaths.split(',')]

                with open('data.csv', 'w') as fh:
                    fh.write('date,number of daily cases,number of daily deaths\n')
                    for i in range(len(dates)):
                        fh.write(f'{dates[i]},{cases[i]},{deaths[i]}\n')
                        
                parsed = True
                bot.send_message(message.from_user.id, "Парсинг успешно закончен. Выберите следующую команду:")
                bot.send_message(message.from_user.id, "/file - Получить файл с данными\
                \n/median - Посчитать медиану. После команды через пробел напишите название колонки, для которой нужно найти медиану (cases или deaths)\
                \n/mean - Посчитать среднее. После команды через пробел напишите название колонки, для которой нужно найти среднее (cases или deaths)\
                \n/date - Получить информацию по конкретному дню. После команды через пробел месяц и день в формате Feb 15. Даты меньше 10 с ведушим нулем - 01,02")

            except Exception:
                parsed = False
                bot.send_message(message.from_user.id, "Произошла ошибка при парсинге. Попробуйте снова или смените страну.")
        else:
            show_parse_help(message)
            parse(message)
    else:
        bot.send_message(message.from_user.id, "Команда не распознана.")
        bot.send_message(message.from_user.id, "/file - Получить файл с данными\
                \n/median - Посчитать медиану. После команды через пробел напишите номер колонки для которой нужно найти медиану\
                \n/mean - Посчитать среднее. После команды через пробел напишите номер колонки для которой нужно найти медиану\
                \n/date - Получить информацию по конкретному дню. После команды через пробел месяц и день в формате Feb 15. Даты меньше 10 с ведушим нулем - 01,02")

bot.polling(none_stop=True, interval=0)